In [1]:
!pip install accelerate peft bitsandbytes transformers trl

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda115.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 115
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda115.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/home/data/thesis/Untitled.ipynb')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inlin

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [29]:
def load_model(model_name, bnb_config):
    n_gpus = torch.cuda.device_count()

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        max_memory = {i: f'{40960}MB' for i in range(n_gpus)},
    )

    # Initializing the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token = True)

    # Needed for LLaMA tokenizer
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

def create_bnb_config():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    return bnb_config
    
def create_peft_config(modules):
    """
    Create Parameter-Efficient Fine-Tuning config for your model
    :param modules: Names of the modules to apply Lora to
    """
    config = LoraConfig(
        r=16,  # dimension of the updated matrices
        lora_alpha=64,  # parameter for scaling
        target_modules=modules,
        lora_dropout=0.1,
        task_type="CAUSAL_LM",
    )

    return config

def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

# Loading Model and Tokenizer

In [11]:
model, tokenizer = load_model("elliotthwang/elliott_Llama-2-7b-hf" , create_bnb_config())

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.72s/it]


In [12]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel


peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_params)

# Loading the Dataset

In [24]:
import pandas as pd

dataset = load_dataset("csv", data_files="ielts_chat/datasets/ielts_buddy_dataset.csv", split='train')
print(len(dataset))

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 624.62it/s]
                                                        

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d5eaa2522004f691/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.
42


In [25]:
def prompt_fn(sample):    
    prompt = """
you are an IELTS examiner. your task is to evaluate a writing section in an IELTS academic
exam. you have to provide overall band score in <BAND_SCORE> </BAND_SCORE> tags and detailed evaluation in <EVALUATION></EVALUATION> tags . I will provide you the grading
criteria in <CRITERIA> </CRITERIA> tags. The user will send you the task and his answer and you should respond with a feedback on how well does the user follow the grading criteria and his score. Provide his score in this format <Score>Score</Score>.
<CRITERIA>
TASK RESPONSE (TR)
For Task 2 of both AC and GT Writing tests, candidates are required to formulate and
develop a position in relation to a given prompt in the form of a question or
statement, using a minimum of 250 words. Ideas should be supported by evidence,
and examples may be drawn from a candidate’s own experience.
The TR criterion assesses:
▪ how fully the candidate responds to the task.
▪ how adequately the main ideas are extended and supported.
▪ how relevant the candidate’s ideas are to the task.
▪ how clearly the candidate opens the discourse, establishes their position and
formulates conclusions.
▪ how appropriate the format of the response is to the task.
COHERENCE AND COHESION (CC)
This criterion is concerned with the overall organisation and logical development of
the message: how the response organises and links information, ideas and language.
Coherence refers to the linking of ideas through logical sequencing, while cohesion
refers to the varied and appropriate use of cohesive devices (e.g. logical connectors,
conjunctions and pronouns) to assist in making clear the relationships between and
within sentences.
The CC criterion assesses:
▪ the coherence of the response via the logical organisation of information
and/or ideas, or the logical progression of the argument.
▪ the appropriate use of paragraphing for topic organisation and presentation.
▪ the logical sequencing of ideas and/or information within and across
paragraphs.
▪ the flexible use of reference and substitution (e.g. definite articles, pronouns).
▪ the appropriate use of discourse markers to clearly mark the stages in a
response, e.g. [First of all | In conclusion], and to signal the relationship between ideas and/or information, e.g. [as a result | similarly].

LEXICAL RESOURCE (LR)
This criterion refers to the range of vocabulary the candidate has used and the
accuracy and appropriacy of that use in terms of the specific task.
The LR criterion assesses:
▪ the range of general words used (e.g. the use of synonyms to avoid repetition).
▪ the adequacy and appropriacy of the vocabulary (e.g. topic-specific items,
indicators of writer’s attitude).
▪ the precision of word choice and expression.
▪ the control and use of collocations, idiomatic expressions and sophisticated
phrasing.
▪ the density and communicative effect of errors in spelling.
▪ the density and communicative effect of errors in word formation.
GRAMMATICAL RANGE AND ACCURACY (GRA)
This criterion refers to the range and accurate use of the candidate’s grammatical
resource via the candidate’s writing at sentence level.
The GRA criterion assesses:
▪ the range and appropriacy of structures used in a given response (e.g. simple,
compound and complex sentences).
▪ the accuracy of simple, compound and complex sentences.
▪ the density and communicative effect of grammatical errors.
▪ the accurate and appropriate use of punctuation.
</CRITERIA>
"""
    
    prompt += f"Here is the task:\n <Task>{sample['Question']}</Task> \n And here is my answer: \n <Answer>{sample['Answer']}</Answer>"
            
    sample['text'] = prompt
    
    return sample

In [28]:
# Add prompt to each sample
print("Preprocessing dataset...")
dataset = dataset.map(prompt_fn)

print(f'Column names are: {dataset.column_names}')

Preprocessing dataset...


Column names are: ['Question', 'Answer', 'Feedback', 'Final Score', 'Unnamed: 4', 'text']

you are an IELTS examiner. your task is to evaluate a writing section in an IELTS academic
exam. you have to provide overall band score in <BAND_SCORE> </BAND_SCORE> tags and detailed evaluation in <EVALUATION></EVALUATION> tags . I will provide you the grading
criteria in <CRITERIA> </CRITERIA> tags. The user will send you the task and his answer and you should respond with a feedback on how well does the user follow the grading criteria and his score. Provide his score in this format <Score>Score</Score>.
<CRITERIA>
TASK RESPONSE (TR)
For Task 2 of both AC and GT Writing tests, candidates are required to formulate and
develop a position in relation to a given prompt in the form of a question or
statement, using a minimum of 250 words. Ideas should be supported by evidence,
and examples may be drawn from a candidate’s own experience.
The TR criterion assesses:
▪ how fully the candidate responds 

In [32]:
from functools import partial

dataset = dataset.map(
        partial(preprocess_batch, max_length=1024, tokenizer=tokenizer),
        batched=True,
        remove_columns=['Question', 'Answer', 'Feedback', 'Final Score', 'Unnamed: 4', 'text']
    )

print(f'Column names are: {dataset.column_names}')

Column names are: ['input_ids', 'attention_mask']


In [39]:
train_val = dataset.train_test_split(test_size=0.11, shuffle=True, seed=42)
train_data = train_val["train"]
val_data = train_val["test"]

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/csv/default-d5eaa2522004f691/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-71eef5039a117af9.arrow and /root/.cache/huggingface/datasets/csv/default-d5eaa2522004f691/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-5154a7ef6e59e88e.arrow


# Training

In [44]:
from transformers import Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=5,
    logging_steps=5,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to= None)

  # Training parameters
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    tokenizer=tokenizer)

trainer.train()

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.


Step,Training Loss
5,0.735800
10,0.332500
15,0.177100
20,0.132300


TrainOutput(global_step=20, training_loss=0.3444117307662964, metrics={'train_runtime': 49.2487, 'train_samples_per_second': 1.503, 'train_steps_per_second': 0.406, 'total_flos': 3019324142911488.0, 'train_loss': 0.3444117307662964, 'epoch': 2.0})